In [7]:
#!pip install tensorflow

In [134]:
import tensorflow as tf

import numpy as np
import os
import time

In [135]:
# Read, then decode for py2 compat.
text = open('corpus.txt','rb').read().decode(encoding='utf-8')

In [136]:
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 25439 characters


In [137]:
# Take a look at the first 250 characters in text
print(text[:250])

One winter's evening, towards the close of the year 1800, or within a year
or two of that time, a young medical practitioner, recently established in
business, was seated by a cheerful fire in his little parlour, listening to the
wind which was be


In [138]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


In [139]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [140]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  "'" :   4,
  '(' :   5,
  ')' :   6,
  ',' :   7,
  '-' :   8,
  '.' :   9,
  '0' :  10,
  '1' :  11,
  '8' :  12,
  ':' :  13,
  ';' :  14,
  '?' :  15,
  'A' :  16,
  'B' :  17,
  'C' :  18,
  'D' :  19,
  ...
}


In [141]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

"One winter's " ---- characters mapped to int ---- > [29 52 43  2 61 47 52 58 43 56  4 57  2]


In [142]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

O
n
e
 
w


In [143]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

"One winter's evening, towards the close of the year 1800, or within a year\r\nor two of that time, a yo"
'ung medical practitioner, recently established in\r\nbusiness, was seated by a cheerful fire in his lit'
'tle parlour, listening to the\r\nwind which was beating the rain in pattering drops against the window,'
' or\r\nrumbling dismally in the chimney. The night was wet and cold; he had\r\nbeen walking through mud a'
'nd water the whole day, and was now\r\ncomfortably reposing in his dressing-gown and slippers, more tha'


In [144]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [145]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "One winter's evening, towards the close of the year 1800, or within a year\r\nor two of that time, a y"
Target data: "ne winter's evening, towards the close of the year 1800, or within a year\r\nor two of that time, a yo"


In [146]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 29 ('O')
  expected output: 52 ('n')
Step    1
  input: 52 ('n')
  expected output: 43 ('e')
Step    2
  input: 43 ('e')
  expected output: 2 (' ')
Step    3
  input: 2 (' ')
  expected output: 61 ('w')
Step    4
  input: 61 ('w')
  expected output: 47 ('i')


In [147]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [148]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [149]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [150]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [151]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [152]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
gru_6 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_6 (Dense)              (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [153]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [154]:
sampled_indices

array([ 6, 62, 60, 27,  6, 27, 64, 17, 53,  8,  5, 63, 33, 38, 16, 18, 11,
       37,  1, 38, 62, 36, 44, 61, 25, 21, 46, 20, 11, 46, 35, 47, 57, 58,
       59, 14, 46, 54, 19, 63,  0,  3, 19, 36, 19, 60, 52, 57, 51, 28, 61,
       18,  8,  0, 49, 17, 51, 34, 63,  1,  8, 63, 17, 17, 12, 51, 30,  1,
       41, 41,  6, 59,  6, 51,  2, 64, 54, 11, 56, 33, 24, 60, 46, 41,  8,
        1, 43,  6, 10, 16,  7, 15, 43, 37, 14, 36,  3,  2, 36, 19])

In [155]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '; and, now and then, a miserable patch of garden-ground,\r\nwith a few old boards knocked together for'

Next Char Predictions: 
 ')xvM)MzBo-(yT_AC1Y\r_xWfwKFhE1hVistu;hpDy\n!DWDvnsmNwC-\nkBmUy\r-yBB8mP\rcc)u)m zp1rTIvhc-\re)0A,?eY;W! WD'


In [156]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1737595


In [157]:
model.compile(optimizer='adam', loss=loss)

In [158]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [159]:
EPOCHS=50

In [160]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 3 steps
Epoch 1/50
3/3 [==============================] - 6s 2s/step - loss: 4.1229
Epoch 2/50
3/3 [==============================] - 6s 2s/step - loss: 4.7004
Epoch 3/50
3/3 [==============================] - 6s 2s/step - loss: 3.6706
Epoch 4/50
3/3 [==============================] - 5s 2s/step - loss: 3.6113
Epoch 5/50
3/3 [==============================] - 5s 2s/step - loss: 3.4851
Epoch 6/50
3/3 [==============================] - 5s 2s/step - loss: 3.3605
Epoch 7/50
3/3 [==============================] - 5s 2s/step - loss: 3.1765
Epoch 8/50
3/3 [==============================] - 5s 2s/step - loss: 3.0148
Epoch 9/50
3/3 [==============================] - 5s 2s/step - loss: 3.0000
Epoch 10/50
3/3 [==============================] - 5s 2s/step - loss: 2.9265
Epoch 11/50
3/3 [==============================] - 6s 2s/step - loss: 2.8678
Epoch 12/50
3/3 [==============================] - 6s 2s/step - loss: 2.8282
Epoch 13/50
3/3 [==============================] - 6s 2s/step - los

In [161]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_50'

In [162]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [163]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_7 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_7 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [215]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 5

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [216]:
print(generate_text(model, start_string=u"The sky is"))

The sky is caig


In [217]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [218]:
optimizer = tf.keras.optimizers.Adam()

In [219]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [220]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # initializing the hidden state at the start of every epoch
  # initally hidden is None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.175134658813477
Epoch 1 Loss 4.0693
Time taken for 1 epoch 6.118037939071655 sec

Epoch 2 Batch 0 Loss 3.8320510387420654
Epoch 2 Loss 3.5987
Time taken for 1 epoch 5.000967025756836 sec

Epoch 3 Batch 0 Loss 3.8342347145080566
Epoch 3 Loss 3.9507
Time taken for 1 epoch 5.501431703567505 sec

Epoch 4 Batch 0 Loss 3.955986261367798
Epoch 4 Loss 3.9279
Time taken for 1 epoch 5.202840566635132 sec

Epoch 5 Batch 0 Loss 3.906343936920166
Epoch 5 Loss 3.8335
Time taken for 1 epoch 5.206571102142334 sec

Epoch 6 Batch 0 Loss 3.7908694744110107
Epoch 6 Loss 3.6584
Time taken for 1 epoch 5.032634735107422 sec

Epoch 7 Batch 0 Loss 3.572174072265625
Epoch 7 Loss 3.3720
Time taken for 1 epoch 5.031881809234619 sec

Epoch 8 Batch 0 Loss 3.3343067169189453
Epoch 8 Loss 3.1891
Time taken for 1 epoch 5.054827928543091 sec

Epoch 9 Batch 0 Loss 3.074802875518799
Epoch 9 Loss 2.9634
Time taken for 1 epoch 5.0290751457214355 sec

Epoch 10 Batch 0 Loss 3.0132524967193604
Epoch 10 